In [4]:
base = '/u/ebanner/Classes/nlp/Project/irony-classifier'

In [ ]:
base = '/u/npockrus/NLP/finalProject/venv/src/irony-classifier'

In [5]:
cd /{base}/data/conservative/features/text+sentiment+subreddit+label/

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/conservative/features/text+sentiment+subreddit+label


# Load Comments 

In [6]:
import pickle
import scipy
import numpy as np

with open('features.p', 'r') as f:
    data = pickle.load(f)

subreddits = np.array([data[sentence]['subreddits'] for sentence in sorted(data)])
sentiments = np.array([data[sentence]['sentiment'] for sentence in sorted(data)])
punctuations = np.array([data[sentence]['punctuation'] for sentence in sorted(data)])
xs = np.array([ sentence.encode('utf-8') for sentence in sorted(data) ])
ys = np.array([ data[sentence]['label'] for sentence in sorted(data) ])

In [7]:
# concatenate features
features = [ x + ' ' + punct + ' ' + (' SUBREDDITx'.join(subreddit_list)).encode('utf-8') for subreddit_list, x, punct in zip(subreddits, xs, punctuations)]

#Uncomment this and comment the one above to reproduce byron's results
#features = [ x + ' ' + punct for x, punct in zip(xs, punctuations)]

print len(xs)
    

2748


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=50000, binary=True, stop_words="english")
#print subreddits
X = vectorizer.fit_transform(features)

In [9]:
print len(xs)
print len(ys)
vectorizer.get_feature_names()

2748
2748


[u'000',
 u'02',
 u'03',
 u'04',
 u'040',
 u'05',
 u'050',
 u'07',
 u'08',
 u'09',
 u'10',
 u'100',
 u'100k',
 u'11',
 u'118',
 u'11e2',
 u'11was',
 u'12',
 u'120',
 u'1200',
 u'1297',
 u'13',
 u'1342',
 u'135',
 u'142',
 u'14pm',
 u'15',
 u'150',
 u'169',
 u'17',
 u'172',
 u'175',
 u'176565745',
 u'17th',
 u'18',
 u'19',
 u'190',
 u'1940',
 u'1944',
 u'1948',
 u'1952',
 u'1968',
 u'1982',
 u'1991',
 u'1995',
 u'1pfyb2',
 u'1rlvxk',
 u'1st',
 u'1trillion',
 u'20',
 u'200',
 u'2002',
 u'2007',
 u'2008',
 u'2009',
 u'200k',
 u'2010',
 u'2011',
 u'2012',
 u'2013',
 u'2014',
 u'2016',
 u'2020',
 u'204',
 u'21213547',
 u'22',
 u'222249311',
 u'228',
 u'229635361',
 u'23',
 u'246',
 u'25',
 u'250',
 u'2500',
 u'255',
 u'26',
 u'26fe',
 u'27',
 u'28',
 u'29',
 u'2k',
 u'2nd',
 u'30',
 u'300',
 u'3000',
 u'31',
 u'33',
 u'339',
 u'34',
 u'3403',
 u'350',
 u'359957',
 u'37',
 u'3x',
 u'40',
 u'401307114770931712',
 u'404',
 u'41',
 u'42',
 u'420',
 u'426',
 u'429',
 u'43554480',
 u'44173',
 u'4

# 5-Fold Test

In [10]:
from sklearn.cross_validation import KFold

kf = KFold(len(xs), n_folds=5, shuffle=True)

# Predictions

In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

recalls, precisions, f_measures = [], [], []
for train, test in kf:
    # Get training and test data for this round
    Xs_train, Xs_test = X[train], X[test]
    ys_train, ys_test = ys[train], ys[test]

    print Xs_train
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto", alpha=.01)
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(Xs_train, ys_train)

    # Make predictions
    predictions = clf.predict(Xs_test)
    
    for test, vectaor, prediction, actual in zip(xs[test], Xs_test, predictions, ys_test):
        print '========================================================================'
        print
        print 'Comment: {}'.format(test)
        print 'Vectorized as: {}'.format(vector)
        print 'Prediction: {}'.format(prediction)
        print 'Was really: {}'.format(actual)
        print

  (0, 4816)	1
  (0, 4907)	1
  (0, 4935)	1
  (0, 4741)	1
  (0, 5035)	1
  (0, 4893)	1
  (0, 4885)	1
  (0, 4884)	1
  (0, 4603)	1
  (0, 5031)	1
  (0, 4761)	1
  (0, 4690)	1
  (0, 4941)	1
  (0, 4717)	1
  (0, 4911)	1
  (0, 3173)	1
  (0, 4134)	1
  (0, 3400)	1
  (0, 1005)	1
  (0, 1262)	1
  (0, 3272)	1
  (1, 4801)	1
  (1, 5016)	1
  (1, 4790)	1
  (1, 4903)	1
  :	:
  (2197, 3956)	1
  (2197, 5574)	1
  (2197, 4636)	1
  (2197, 5004)	1
  (2197, 4897)	1
  (2197, 5022)	1
  (2197, 4608)	1
  (2197, 4975)	1
  (2197, 4944)	1
  (2197, 2784)	1
  (2197, 4767)	1
  (2197, 4672)	1
  (2197, 446)	1
  (2197, 4781)	1
  (2197, 4801)	1
  (2197, 4623)	1
  (2197, 4771)	1
  (2197, 4980)	1
  (2197, 4993)	1
  (2197, 4907)	1
  (2197, 5035)	1
  (2197, 5031)	1
  (2197, 4761)	1
  (2197, 4941)	1
  (2197, 4911)	1

Comment: 


So when CEO's do it, it's wrong and evil, but when the fed does it to keep Obama and the democrats from looking bad spending all this money it's swept under the bus lol



NameError: name 'vector' is not defined

# Most Indicative Negative Features

In [15]:
sgd_clf = clf.best_estimator_
feat_names = vectorizer.get_feature_names()
coefs_with_fns = sorted(zip(sgd_clf.coef_[0], feat_names))
coefs_with_fns[:20]

[(-0.1138284885109184, u'subredditxscience'),
 (-0.10237446581940357, u'subredditxpolitics'),
 (-0.081063693831936373, u'subredditxiama'),
 (-0.050814811651664855, u'subredditxbooks'),
 (-0.04670990178534154, u'atheism'),
 (-0.045518153759634833, u'subredditxliberal'),
 (-0.042431195331935942, u'subredditxprogressive'),
 (-0.041330483613748491, u'subredditxfood'),
 (-0.036778668237785077, u'subredditxentertainment'),
 (-0.036166242169019001, u'subredditxoffbeat'),
 (-0.034974494143312274, u'subredditxfantasyfootball'),
 (-0.034873526602244957, u'subredditxjusticeporn'),
 (-0.034693109192798187, u'subredditxscifi'),
 (-0.032499961784379472, u'subredditxdemocrats'),
 (-0.032144092582258685, u'subredditxmorbidreality'),
 (-0.031142693199546725, u'subredditxstlouis'),
 (-0.031101313059765396, u'subredditxfitness'),
 (-0.027219855948261901, u'bodyacceptance'),
 (-0.027219855948261901, u'subredditxbooksuggestions'),
 (-0.027219855948261901, u'subredditxdmb')]

# Most Indicative Features

In [18]:
coefs_with_fns[-1:-20:-1]

[(0.12176023370421923, u'subredditxadviceanimals'),
 (0.096197238552808331, u'politics'),
 (0.085188466165350321, u'puncxexclamation_point'),
 (0.080042431982124337, u'subredditxwtf'),
 (0.07421445309530024, u'puncxquestion_mark'),
 (0.058286409690610781, u'subredditxskyrim'),
 (0.058286409690610781, u'subredditxgaming4gamers'),
 (0.057791503218827915, u'subredditxatheism'),
 (0.053186721263943283, u'subredditxvideos'),
 (0.048224414901346248, u'subredditxaww'),
 (0.047037632492415195, u'subredditxfunny'),
 (0.042868169608032088, u'subredditxgaming'),
 (0.039081059215227275, u'just'),
 (0.039067817570497335, u'subredditxdoesanybodyelse'),
 (0.038885744955458747, u'subredditxgadgets'),
 (0.037887655983929201, u'subredditxhomeimprovement'),
 (0.034072407096076111, u'subredditxmapporn'),
 (0.033820815846205367, u'subredditxpics'),
 (0.033815850229430851, u'obamacare')]

# Performance

## Byron's Results: Precision=0.315 (0.261, 0.380), Recall=0.496 (0.446, 0.548), F-Measure=0.383 (0.330, 0.412)

In [44]:
import sklearn

precisions, recalls, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto")
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    # Record statistics
    precision, recall, f_measure, _ = sklearn.metrics.precision_recall_fscore_support(ys[test], predictions, average='binary')
    precisions.append(precision)
    recalls.append(recall)
    f_measures.append(f_measure)
    
print 'Precisions: {}'.format(precisions)
print 'Recalls: {}'.format(recalls)
print 'F-Measures: {}'.format(f_measures)
print
print 'Mean Precision: {}'.format(np.mean(precisions))
print 'Mean Recall: {}'.format(np.mean(recalls))
print 'Mean F-Measure: {}'.format(np.mean(f_measures))

Precisions: [0.091428571428571428, 0.058823529411764705, 0.083333333333333329, 0.090909090909090912, 0.069204152249134954]
Recalls: [0.47058823529411764, 0.88, 0.25925925925925924, 0.5, 0.7142857142857143]
F-Measures: [0.15311004784688997, 0.11027568922305764, 0.12612612612612614, 0.15384615384615385, 0.12618296529968454]

Mean Precision: 0.0787397354664
Mean Recall: 0.564826641768
Mean F-Measure: 0.133908196468
